# Início

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm
import time
import os
from scipy import signal
from scipy.signal import find_peaks
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import pickle

In [2]:
with open('Udata.pkl', 'rb') as file:
    Udata = pickle.load(file)

In [3]:
# Vamos remover a atividade 'jumping'
actis = ['climbingdown', 'climbingup', 'lying', 'running', 'sitting', 'standing', 'walking']
posis = ['chest', 'forearm', 'head', 'shin', 'thigh', 'upperarm', 'waist']
users = ['proband' + x for x in np.arange(1,16).astype(str)]
# proband2 não tem acc_climbingup_forearm
users.remove('proband2')
# Vamos remover usuários com menos de 21000 amostras
users.remove('proband1')
users.remove('proband4')
users.remove('proband7')
users.remove('proband14')

In [4]:
# Udata está organizado como [usuario][posicao][atividade][tempo][dims]
Dtr = np.array(Udata[5][0])[:, :11000, :]
Xtr = Dtr.reshape(-1).reshape(770, 600)

In [5]:
Xtr.shape

(770, 600)

In [6]:
Dtr.shape

(7, 11000, 6)

In [9]:
px.line(Dtr[2])

In [44]:
def plotCR(X):
    N = X.shape[0]
    L = N*(N-1)//2
    rx = np.zeros(L)
    cont = 0
    for i in range(N-1):
        for j in range(i+1,N):
            rx[cont] = max(abs(X[i,:]-X[j,:]))*2
            cont += 1
    rx = np.sort(rx)
    ex = np.log2(rx)
    ey = np.log2(np.arange(1,L+1)/L)
    px.scatter(x=ex,y=ey)

In [43]:
X = np.array(Udata[5][0])[0, 1000:1100, :]
N = X.shape[0]
L = N*(N-1)//2
rx = np.zeros(L)
cont = 0
for i in range(N-1):
    for j in range(i+1,N):
        rx[cont] = max(abs(X[i,:]-X[j,:]))*2
        cont += 1
rx = np.sort(rx)
ex = np.log2(rx)
ey = np.log2(np.arange(1,L+1)/L)
px.scatter(x=ex,y=ey)